# `VKS document with llama3`


In [1]:
%load_ext autoreload
%autoreload 2

## On its own


In [2]:
from haystack import Pipeline
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack.components.converters import PDFMinerToDocument
from haystack.components.preprocessors import DocumentCleaner
from haystack.components.preprocessors import DocumentSplitter
from haystack.components.writers import DocumentWriter

In [3]:
raw_document_store = InMemoryDocumentStore(embedding_similarity_function="cosine")

pipeline = Pipeline()
pipeline.add_component("converter", PDFMinerToDocument())
pipeline.add_component("cleaner", DocumentCleaner())
pipeline.add_component("splitter", DocumentSplitter(split_by="sentence", split_length=20, split_overlap=15))
pipeline.add_component("writer", DocumentWriter(document_store=raw_document_store))

pipeline.connect("converter", "cleaner")
pipeline.connect("cleaner", "splitter")
pipeline.connect("splitter", "writer")

🚅 Components
  - converter: PDFMinerToDocument
  - cleaner: DocumentCleaner
  - splitter: DocumentSplitter
  - writer: DocumentWriter
🛤️ Connections
  - converter.documents -> cleaner.documents (List[Document])
  - cleaner.documents -> splitter.documents (List[Document])
  - splitter.documents -> writer.documents (List[Document])

In [4]:
file_names = ["sample.pdf"]
pipeline.run({"converter": {"sources": file_names}})

{'writer': {'documents_written': 687}}

In [5]:
from haystack.components.embedders import SentenceTransformersDocumentEmbedder

In [6]:
document_store = InMemoryDocumentStore(embedding_similarity_function="cosine")

Initalize a Document Embedder

In [7]:
doc_embedder = SentenceTransformersDocumentEmbedder(model="sentence-transformers/all-MiniLM-L6-v2")
doc_embedder.warm_up()

In [8]:
docs_with_embeddings = doc_embedder.run(raw_document_store.filter_documents())
document_store.write_documents(docs_with_embeddings["documents"])

Batches:   0%|          | 0/22 [00:00<?, ?it/s]

687

In [9]:
from haystack.components.embedders import SentenceTransformersTextEmbedder
from haystack.components.builders import ChatPromptBuilder
from haystack.dataclasses import ChatMessage
from haystack_integrations.components.generators.ollama import OllamaChatGenerator
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever

In [10]:
text_embedder = SentenceTransformersTextEmbedder(model="sentence-transformers/all-MiniLM-L6-v2")

In [11]:
template = [ChatMessage.from_user("""
Given the following information, answer the question.

Context:
{% for document in documents %}
    {{ document.content }}
{% endfor %}

Question: {{question}}
Answer:
""")]

prompt_builder = ChatPromptBuilder(template=template)

In [12]:
chat_generator = OllamaChatGenerator(
    model="llama3.1:8b",
    streaming_callback=lambda chunk: print(chunk.content, end="", flush=True),
    url = "http://localhost:11434",
    generation_kwargs={
        "num_predict": 1024,
        "temperature": 0.01})

Initialize retriever

In [13]:
retriever = InMemoryEmbeddingRetriever(document_store)

In [14]:
basic_rag_pipeline = Pipeline()
# Add components to your pipeline
basic_rag_pipeline.add_component("text_embedder", text_embedder)
basic_rag_pipeline.add_component("retriever", retriever)
basic_rag_pipeline.add_component("prompt_builder", prompt_builder)
basic_rag_pipeline.add_component("llm", chat_generator)

In [15]:
# Now, connect the components to each other
basic_rag_pipeline.connect("text_embedder.embedding", "retriever.query_embedding")
basic_rag_pipeline.connect("retriever", "prompt_builder")
basic_rag_pipeline.connect("prompt_builder.prompt", "llm.messages")

🚅 Components
  - text_embedder: SentenceTransformersTextEmbedder
  - retriever: InMemoryEmbeddingRetriever
  - prompt_builder: ChatPromptBuilder
  - llm: OllamaChatGenerator
🛤️ Connections
  - text_embedder.embedding -> retriever.query_embedding (List[float])
  - retriever.documents -> prompt_builder.documents (List[Document])
  - prompt_builder.prompt -> llm.messages (List[ChatMessage])

# Asking question

In [16]:
question = "What is VKS?"

response = basic_rag_pipeline.run({"text_embedder": {"text": question}, "prompt_builder": {"question": question}})

print(f"\n\n\n\n\n==========================\nAnswer: {response["llm"]["replies"][0].text}")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

VKS stands for VNGCloud Kubernetes Service. It appears to be a cloud-based platform that provides a managed Kubernetes service, allowing users to create and manage clusters, nodes, volumes, load balancers, and other resources in a scalable and secure manner. VKS seems to offer various features such as resource monitoring, event history, automatic garbage collection of unused containers and images, and improved performance.




Answer: VKS stands for VNGCloud Kubernetes Service. It appears to be a cloud-based platform that provides a managed Kubernetes service, allowing users to create and manage clusters, nodes, volumes, load balancers, and other resources in a scalable and secure manner. VKS seems to offer various features such as resource monitoring, event history, automatic garbage collection of unused containers and images, and improved performance.


In [17]:
question = "Compare VKS private clusters and VKS public clusters"

response = basic_rag_pipeline.run({"text_embedder": {"text": question}, "prompt_builder": {"question": question}})

print(f"\n\n\n\n\n\n\n================\nAnswer: {response["llm"]["replies"][0].text}")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Here's a comparison of VKS Private Clusters and VKS Public Clusters:

**Key Differences:**

* **Security:** Private Clusters have higher security with all connections within VNG Cloud's private network, minimizing the risk of external network attacks. Public Clusters have medium security since they use public IP addresses.
* **Access Management:** Private Clusters have strict access control, while Public Clusters require more complex configuration and management for access control.
* **Scalability:** Both types of clusters are easily scalable through Auto Scaling feature.
* **Accessibility:** Public Clusters can be accessed from anywhere with internet, while Private Clusters must be accessed through other security solutions.
* **Configuration and Deployment:** Private Clusters require more complex configuration and deployment due to their private and secure network setup.
* **Cost:** Private Clusters have higher costs due to additional security and management components required.

**Wh

In [18]:
question = "What is vngcloud-blockstorage-csi-driver?"

response = basic_rag_pipeline.run({"text_embedder": {"text": question}, "prompt_builder": {"question": question}})

print(f"\n\n\n\n\n\n\n====================\nAnswer: {response["llm"]["replies"][0].text}")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

vngcloud-blockstorage-csi-driver is a CSI (Container Storage Interface) driver provided by VNG Cloud, which allows attaching BlockStorage volumes to Kubernetes pods. It supports ReadWriteMany access, allowing multiple nodes to read and write on the same volume simultaneously.






Answer: vngcloud-blockstorage-csi-driver is a CSI (Container Storage Interface) driver provided by VNG Cloud, which allows attaching BlockStorage volumes to Kubernetes pods. It supports ReadWriteMany access, allowing multiple nodes to read and write on the same volume simultaneously.


In [19]:
question = "I want to create a pvc of 20Gi using vngcloud-blockstorage-csi-driver"

response = basic_rag_pipeline.run({"text_embedder": {"text": question}, "prompt_builder": {"question": question}})

print(f"\n\n\n\n\n==============================\nAnswer: {response["llm"]["replies"][0].text}")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

To create a Persistent Volume Claim (PVC) of 20Gi using the `vngcloud-blockstorage-csi-driver`, you will need to follow these steps:

1. Create a YAML file, e.g., `pvc.yaml`, with the following content:
```yaml
apiVersion: v1
kind: PersistentVolumeClaim
metadata:
  name: my-pvc
spec:
  accessModes:
    - ReadWriteOnce
  resources:
    requests:
      storage: 20Gi
```
This YAML file defines a PVC named `my-pvc` that requests 20Gi of storage.

2. Apply the YAML file to your Kubernetes cluster using the following command:
```bash
kubectl apply -f pvc.yaml
```
3. Verify that the PVC has been created successfully by running the following command:
```bash
kubectl get pvc my-pvc
```
This should display information about the PVC, including its status.

4. To use the PVC with a pod, you will need to create a Deployment or Pod that references the PVC. For example, you can add a volume to your `nginx-service.yaml` file:
```yaml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: nginx-app
spe

In [20]:
question = "How to use volume snapshot in VKS cluster"

response = basic_rag_pipeline.run({"text_embedder": {"text": question}, "prompt_builder": {"question": question}})

print(f"\n\n\n===========================\nAnswer: {response["llm"]["replies"][0].text}")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

To use volume snapshots in a VKS (VNGCloud Snapshot) cluster, you can follow these steps:

1. **Enable the Snapshot service**: Visit https://hcm-3.console.vngcloud.vn/vserver/block-store/snapshot/overview and select "Activate Snapshot Service".
2. **Install VNGCloud Snapshot Controller**: Install Helm version 3.0 or higher, add the `vks-helm-charts` repository to your cluster, and install the `vngcloud-snapshot-controller` chart using Helm.
3. **Create a VolumeSnapshotClass**: Create a YAML file (e.g., `snapshot.yaml`) with the following content:
```yaml
apiVersion: snapshot.storage.k8s.io/v1
kind: VolumeSnapshotClass
metadata:
  name: my-snapshot-storage-class
driver: bs.csi.vngcloud.vn
deletionPolicy: Delete
parameters:
  force-create: "false"
```
4. **Apply the YAML file**: Run `kubectl apply -f snapshot.yaml` to create the VolumeSnapshotClass.
5. **Create a PersistentVolumeClaim (PVC)**: Create a PVC that references the VolumeSnapshotClass you created earlier. For example:
```yaml
